In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [24]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/My Drive/Car Model Classifier/Datasets/Train'
valid_path = '/content/drive/My Drive/Car Model Classifier/Datasets/test'

In [25]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 4d473c1dd8becc155b73f8504c6f6626 so we will re-download the data.
94773248/94765736 [==============================] - 1s 0us/step


In [26]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [27]:
  # useful for getting number of output classes
folders = glob('/content/drive/My Drive/Car Model Classifier/Datasets/Train/*')
folders

['/content/drive/My Drive/Car Model Classifier/Datasets/Train/audi',
 '/content/drive/My Drive/Car Model Classifier/Datasets/Train/mercedes',
 '/content/drive/My Drive/Car Model Classifier/Datasets/Train/lamborghini']

In [28]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [29]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [30]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [31]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [32]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/My Drive/Car Model Classifier/Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 90 images belonging to 3 classes.


In [34]:
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/Car Model Classifier/Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 58 images belonging to 3 classes.


In [35]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=100,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
3/3 [==============================] - 2s 597ms/step - loss: 6.8157 - accuracy: 0.3333 - val_loss: 4.8586 - val_accuracy: 0.5172
Epoch 2/100
3/3 [==============================] - 1s 364ms/step - loss: 4.8677 - accuracy: 0.3889 - val_loss: 7.9464 - val_accuracy: 0.1552
Epoch 3/100
3/3 [==============================] - 1s 334ms/step - loss: 5.0276 - accuracy: 0.3778 - val_loss: 5.7604 - val_accuracy: 0.3276
Epoch 4/100
3/3 [==============================] - 1s 340ms/step - loss: 2.9957 - accuracy: 0.3889 - val_loss: 2.9508 - val_accuracy: 0.5172
Epoch 5/100
3/3 [==============================] - 1s 374ms/step - loss: 2.7678 - accuracy: 0.4333 - val_loss: 3.2934 - val_accuracy: 0.1724
Epoch 6/100
3/3 [==============================] - 1s 361ms/step - loss: 1.4826 - accuracy: 0.5222 - val_loss: 1.8187 - val_accuracy: 0.4483
Epoch 7/100
3/3 [==============================] - 1s 359ms/step - loss: 1.737

In [37]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('/content/drive/My Drive/Car Model Classifier/model_resnet50.h5')